In [1]:
import duckdb  # Importa o DuckDB para manipulação de dados e execução de SQL
import os  # Importa o módulo os para interagir com variáveis de ambiente do sistema
from dotenv import load_dotenv, find_dotenv  # Importa funções para carregar variáveis de ambiente de um arquivo .env
from deltalake import DeltaTable, write_deltalake
# Carrega as variáveis de ambiente definidas no arquivo .env
load_dotenv(find_dotenv())

# Define as credenciais de acesso ao MinIO a partir das variáveis de ambiente
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_MINIO")  # Chave de acesso do MinIO
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY_MINIO")  # Chave secreta do MinIO
HOST_MINIO = os.getenv("HOST_MINIO")  # Host do MinIO

# Conecta ao DuckDB, criando uma instância de conexão
con = duckdb.connect()

# Cria uma secret no DuckDB para acessar o MinIO com as credenciais fornecidas
con.execute(f"""
    CREATE SECRET my_minio_secret (
        TYPE 'S3',
        KEY_ID '{AWS_ACCESS_KEY}',
        SECRET '{AWS_SECRET_KEY}',
        REGION 'us-east-1',
        ENDPOINT '{HOST_MINIO}:9000',
        URL_STYLE 'path',
        USE_SSL false
    );
""")

# Define o caminho de destino no MinIO para os arquivos Parquet
path_minio_landing = 's3://landing/comex'
path_minio_bronze = 's3://bronze/comex/uf'

In [2]:
storage_options = {
    "AWS_ENDPOINT_URL": f"http://{HOST_MINIO}:9000",
    "AWS_REGION": "us-east-1",
    "AWS_ACCESS_KEY_ID": AWS_ACCESS_KEY,
    "AWS_SECRET_ACCESS_KEY": AWS_SECRET_KEY,
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
    "AWS_ALLOW_HTTP": "true"
}

In [3]:
con.sql(f"""
        SELECT 
        *
         from '{path_minio_landing}/UF.parquet';        

        """)

┌───────┬─────────┬──────────────────────────┬──────────────────────────┐
│ CO_UF │  SG_UF  │          NO_UF           │        NO_REGIAO         │
│ int64 │ varchar │         varchar          │         varchar          │
├───────┼─────────┼──────────────────────────┼──────────────────────────┤
│    99 │ ZN      │ Zona Não Declarada       │ REGIAO NAO DECLARADA     │
│    11 │ RO      │ Rondônia                 │ REGIAO NORTE             │
│    12 │ AC      │ Acre                     │ REGIAO NORTE             │
│    13 │ AM      │ Amazonas                 │ REGIAO NORTE             │
│    14 │ RR      │ Roraima                  │ REGIAO NORTE             │
│    15 │ PA      │ Pará                     │ REGIAO NORTE             │
│    16 │ AP      │ Amapá                    │ REGIAO NORTE             │
│    17 │ TO      │ Tocantins                │ REGIAO NORTE             │
│    21 │ MA      │ Maranhão                 │ REGIAO NORDESTE          │
│    22 │ PI      │ Piauí             

In [4]:
df = con.sql(f"""
        SELECT 
        CAST(CO_UF as int) AS cod_estado,
        SG_UF as sigla_estado,
        NO_UF as nome_estado,
        NO_REGIAO as nome_regiao
         from '{path_minio_landing}/UF.parquet';        

        """).to_arrow_table()

In [5]:
df

pyarrow.Table
cod_estado: int32
sigla_estado: string
nome_estado: string
nome_regiao: string
----
cod_estado: [[99,11,12,13,14,...,94,96,97,98,95]]
sigla_estado: [["ZN","RO","AC","AM","RR",...,"CB","MN","RE","ED","ND"]]
nome_estado: [["Zona Não Declarada","Rondônia","Acre","Amazonas","Roraima",...,"Consumo de Bordo","Mercadoria Nacionalizada","Reexportação","Estados Diversos - Café","Não Declarada"]]
nome_regiao: [["REGIAO NAO DECLARADA","REGIAO NORTE","REGIAO NORTE","REGIAO NORTE","REGIAO NORTE",...,"CONSUMO DE BORDO","MERCADORIA NACIONALIZADA","REEXPORTACAO","REGIAO NAO DECLARADA","REGIAO NAO DECLARADA"]]

In [10]:
write_deltalake(
        f'{path_minio_bronze}',
        df,
        storage_options=storage_options,
        
    )

In [6]:
table_path = f'{path_minio_bronze}'

# Conecte à tabela Delta existente
table = DeltaTable(table_path, storage_options=storage_options)



In [7]:
table.merge(
    source=df,
    predicate='target.cod_estado = source.cod_estado',
    source_alias="source",
    target_alias="target",
).when_not_matched_insert_all().execute()


{'num_source_rows': 34,
 'num_target_rows_inserted': 15,
 'num_target_rows_updated': 0,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 15,
 'num_target_files_added': 1,
 'num_target_files_removed': 0,
 'execution_time_ms': 98,
 'scan_time_ms': 0,
 'rewrite_time_ms': 49}

In [8]:
con.sql(f"""
        SELECT *
FROM delta_scan('{path_minio_bronze}')
        """)

┌────────────┬──────────────┬──────────────────────────┬──────────────────────────┐
│ cod_estado │ sigla_estado │       nome_estado        │       nome_regiao        │
│   int32    │   varchar    │         varchar          │         varchar          │
├────────────┼──────────────┼──────────────────────────┼──────────────────────────┤
│         99 │ ZN           │ Zona Não Declarada       │ REGIAO NAO DECLARADA     │
│         22 │ PI           │ Piauí                    │ REGIAO NORDESTE          │
│         25 │ PB           │ Paraíba                  │ REGIAO NORDESTE          │
│         41 │ SP           │ São Paulo                │ REGIAO SUDESTE           │
│         11 │ RO           │ Rondônia                 │ REGIAO NORTE             │
│         15 │ PA           │ Pará                     │ REGIAO NORTE             │
│         23 │ CE           │ Ceará                    │ REGIAO NORDESTE          │
│         97 │ RE           │ Reexportação             │ REEXPORTACAO       

In [14]:
con.close()